In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [32]:
from datetime import datetime, date
from ml_collections import ConfigDict
import pandas as pd
import numpy as np
import torch
from tqdm.autonotebook import tqdm

In [5]:
from configs.data_configs.sber_contrastive import data_configs
from configs.model_configs.supervised.sber import model_configs
from src.data_load.dataloader import create_data_loaders, create_test_loader
import src.models.base_models as zoo

In [27]:
dev = "cuda"

In [6]:
dc = data_configs()
mc = model_configs()

In [7]:
dc.train.dropout = 0

In [8]:
dc.train.split_strategy = ConfigDict({"split_strategy": "NoSplit"})
dc.val.split_strategy = ConfigDict({"split_strategy": "NoSplit"})

In [9]:
train_loader, val_loader = create_data_loaders(dc, supervised=False)
test_loader = create_test_loader(dc)

1177it [00:00, 2970.06it/s]
1852it [00:00, 3272.64it/s]


In [21]:
net = getattr(zoo, mc.model_name)(model_conf=mc, data_conf=dc)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [24]:
ckpt = torch.load("ckpt/test_2023-10-24_15:54:18/epoch: 0008 - total_loss: 1.337e+05 - entropy_loss: 0 - loss: -8.054e+04.ckpt")

In [28]:
net.load_state_dict(ckpt["model"])
net = net.to(dev)

In [40]:
torch.set_grad_enabled(False)

In [65]:
def eval_loader(loader):
    embs = []
    rdt = []
    epk_id = []
    
    for batch, _ in tqdm(loader):
        rdt.append(batch.payload["report_date"][:, 0].numpy())
        epk_id.append(batch.payload["epk_id"][:, 0].numpy())
        batch = batch.to(dev)
        emb = net(batch).cpu().numpy()
        embs.append(emb)
    
    df = (
        pd.DataFrame(data=np.concatenate(embs, axis=0))
        .rename(columns=lambda c: f"emb_{c}")
        .assign(
            epk_id=np.concatenate(epk_id),
            report_date=[date.fromordinal(it) for it in np.concatenate(rdt)]
        )
        .set_index(["epk_id", "report_date"])
        .sort_index()
    )
    return df

In [66]:
eval_loader(train_loader)

  0%|          | 0/8 [00:00<?, ?it/s]

emb_0     emb_1     emb_2     emb_3     emb_4  \
epk_id report_date                                                     
1      2021-01-31   0.083846 -1.441497 -2.320467 -0.607797  0.595112   
       2021-02-28  -1.993724 -0.353191 -1.743989  0.224349  0.730936   
       2021-04-30  -0.531881 -1.059343 -1.484411  0.583814  0.491580   
       2021-05-31  -0.996396 -0.923315 -2.576798  0.690249  1.101575   
       2021-07-31  -1.101420 -0.690301 -2.606943 -0.372747  0.126712   
...                      ...       ...       ...       ...       ...   
98     2021-09-30  -0.865960 -0.692477 -0.205554 -0.226851 -0.252627   
       2021-10-31  -0.728378 -0.471789 -0.145597 -0.084460 -0.180582   
       2021-11-30  -0.519649 -0.512236  0.009602 -0.278729  0.028890   
       2021-12-31  -0.544487 -0.732230  0.352827 -0.163775 -0.102362   
99     2021-04-30  -0.454332  0.094496 -2.617297  0.303568  1.025045   

                       emb_5     emb_6     emb_7     emb_8     emb_9  ...  \
epk_id report_date                                                    ...   
1      2021-01-31  -0.073312 -0.236692  0.314021  0.198532 -0.885822  ...   
       2021-02-28  -0.476597  0.413770  0.719771  0.650844 -1.672306  ...   
       2021-04-30   0.673979 -0.661876  0.188581  0.900171 -1.704318  ...   
       2021-05-31   0.140984 -0.639664 -0.398602  1.117632 -1.066802  ...   
       2021-07-31  -0.136348 -0.269561  1.317645 -0.177974 -1.871508  ...   
...                      ...       ...       ...       ...       ...  ...   
98     2021-09-30   1.901833  0.748782 -0.133876 -0.540860 -0.318903  ...   
       2021-10-31   1.967243  0.430921 -0.324548 -0.957117 -0.434893  ...   
       2021-11-30   1.950755  0.618020  0.004521 -0.650729 -0.665028  ...   
       2021-12-31   1.951690  0.672252 -0.277844 -0.711136 -0.398487  ...   
99     2021-04-30  -0.277267 -0.229825 -0.183517  1.117549 -1.360951  ...   

                     emb_118   emb_119   emb_120   emb_121   emb_122  \
epk_id report_date                                                     
1      2021-01-31  -0.366131 -0.524320  0.003273  1.599662  0.246016   
       2021-02-28  -0.420786 -0.404535 -0.045086  0.901148 -0.970704   
       2021-04-30  -0.576345  0.385408  0.828535  0.442854  0.883369   
       2021-05-31  -0.758090  0.777598  0.460894  0.295586  0.040074   
       2021-07-31  -0.846006  0.887242  0.251286  1.299562 -0.480953   
...                      ...       ...       ...       ...       ...   
98     2021-09-30  -0.983138  0.748654  0.216785  0.654192 -0.195211   
       2021-10-31  -1.342489  0.916977  0.278852  0.828842 -0.515340   
       2021-11-30  -1.108914  0.910063  0.236982  1.005893  0.149722   
       2021-12-31  -1.033977  0.639785  0.331711  0.854236  0.004707   
99     2021-04-30   0.295932 -0.206567  1.376286  0.711518 -0.220641   

                     emb_123   emb_124   emb_125   emb_126   emb_127  
epk_id report_date                                                    
1      2021-01-31   0.809038 -0.643937  1.030403  0.124856 -0.244492  
       2021-02-28   0.884286 -0.270603 -0.423720 -0.860225 -0.023758  
       2021-04-30   0.147561 -0.202842  2.504353 -0.782357 -0.310617  
       2021-05-31   0.339069 -1.000889 -0.052126 -0.565955 -0.885213  
       2021-07-31   0.992101 -0.100107  1.686021  0.103772 -0.975346  
...                      ...       ...       ...       ...       ...  
98     2021-09-30   0.419429 -1.686229  0.273740 -0.198751  1.456526  
       2021-10-31  -0.034922 -1.600082 -0.163747  0.069662  1.099143  
       2021-11-30   0.416508 -1.535012  0.162012 -0.222195  1.266060  
       2021-12-31   0.241296 -1.677672  0.046620 -0.224260  1.238138  
99     2021-04-30   0.529267 -0.060498 -0.304533 -1.291996 -0.724538  

[1003 rows x 128 columns]

In [67]:
eval_loader(val_loader)

  0%|          | 0/1 [00:00<?, ?it/s]

emb_0     emb_1     emb_2     emb_3     emb_4  \
epk_id report_date                                                     
1      2021-12-31  -0.525223 -0.017943 -2.026469  0.057868  0.434826   
3      2021-12-31  -0.994239 -1.423033 -1.584398  0.207730  0.512448   
4      2021-08-31  -0.681786 -0.506030 -0.339207  0.157737  0.198169   
5      2021-03-31  -1.469782 -0.562991 -2.126976  0.168726  0.743872   
       2021-08-31  -0.808987 -1.079384 -1.712852 -0.184698  1.360115   
...                      ...       ...       ...       ...       ...   
90     2021-12-31  -1.123061 -0.585484 -0.762189 -0.947464 -0.186241   
94     2021-03-31  -0.920733 -0.147923 -0.931728  0.433547  0.299113   
95     2021-12-31  -0.642682 -0.156450 -2.425078  0.083737  0.485854   
96     2021-07-31  -0.858207 -0.938089 -1.513887 -0.706169  0.260288   
       2021-08-31  -1.096232 -0.210825 -1.501823  0.787121  0.649421   

                       emb_5     emb_6     emb_7     emb_8     emb_9  ...  \
epk_id report_date                                                    ...   
1      2021-12-31  -0.190723 -1.090281  0.324277 -0.198410 -1.203174  ...   
3      2021-12-31   0.242789 -0.170219  1.058489  0.175287 -1.055622  ...   
4      2021-08-31   1.780578  0.132474 -0.349893 -0.233525 -0.671901  ...   
5      2021-03-31   0.265703 -1.215025  0.611671 -0.467215 -1.610408  ...   
       2021-08-31   0.164150 -1.024680  0.221090 -0.446901 -2.238205  ...   
...                      ...       ...       ...       ...       ...  ...   
90     2021-12-31   1.280898 -0.611743  0.279472 -0.023113 -1.326171  ...   
94     2021-03-31   1.531040  0.523184  0.246212 -0.438849 -0.938568  ...   
95     2021-12-31  -0.202322 -1.626827  0.203293 -0.366499 -1.497744  ...   
96     2021-07-31   0.460727 -0.700963  0.066090  0.062495 -1.842235  ...   
       2021-08-31   0.968271 -0.510391 -0.290340 -0.305416 -1.044025  ...   

                     emb_118   emb_119   emb_120   emb_121   emb_122  \
epk_id report_date                                                     
1      2021-12-31  -0.021275 -0.379413  0.189693  1.725714 -0.458592   
3      2021-12-31  -0.542420  0.671646 -0.164177  0.713220 -0.082121   
4      2021-08-31  -1.295901  0.465732  0.590583  0.940632 -0.060697   
5      2021-03-31  -0.773153  0.510799  0.439768  0.951734 -0.105215   
       2021-08-31  -0.870097  0.652368  0.913471  0.472522  0.087349   
...                      ...       ...       ...       ...       ...   
90     2021-12-31  -1.063342 -0.215401  1.301850  0.823924 -0.363685   
94     2021-03-31  -0.969334  0.562649  0.354259  1.196256 -0.316063   
95     2021-12-31  -0.541033  0.790666  0.733394  0.631276 -0.415129   
96     2021-07-31  -0.628892  0.297968  0.034154  1.092522 -0.680828   
       2021-08-31  -0.607154 -0.229334  1.666759  0.562233  0.154142   

                     emb_123   emb_124   emb_125   emb_126   emb_127  
epk_id report_date                                                    
1      2021-12-31   0.835933 -0.161531  1.015627 -0.784544 -0.948462  
3      2021-12-31   0.782185 -0.779837  0.959405 -0.498876 -0.158705  
4      2021-08-31   0.281535 -1.312024  0.048654  0.167500  0.511545  
5      2021-03-31   0.147588 -0.159135  0.718180  0.360093 -0.572816  
       2021-08-31   0.470240 -0.406885  1.390899  0.190110 -0.015987  
...                      ...       ...       ...       ...       ...  
90     2021-12-31   0.139235 -0.696283  1.135142 -1.362365  0.124143  
94     2021-03-31   0.410670 -1.425229  0.923234 -0.677046  0.471399  
95     2021-12-31   0.841496 -0.347798  0.602998  0.395650 -0.585620  
96     2021-07-31   0.387066 -0.546149  1.228905 -1.079253 -0.791308  
       2021-08-31   0.491910 -0.599628  1.167702 -0.321332  0.025919  

[111 rows x 128 columns]

In [68]:
eval_loader(test_loader)

  0%|          | 0/15 [00:00<?, ?it/s]

emb_0     emb_1     emb_2     emb_3     emb_4  \
epk_id report_date                                                     
0      2022-01-31  -1.128832 -0.590986 -3.103005 -0.132528  0.719996   
       2022-02-28  -1.337687 -0.955397 -2.340914 -0.220238 -0.042862   
       2022-03-31   0.496663  0.368241 -2.795825  1.167059  0.283441   
       2022-04-30  -0.968219  0.669099 -2.994418  0.551368  1.594059   
1      2022-01-31  -0.568623 -0.951221 -1.803696  1.034556  0.045360   
...                      ...       ...       ...       ...       ...   
468    2022-04-30  -1.073594 -0.730503 -0.384806  0.379599 -0.004009   
469    2022-01-31  -0.237362 -0.257158 -1.236593 -0.502018  0.357613   
       2022-02-28  -0.457741 -1.346044 -0.828019 -0.002603  0.303320   
       2022-03-31  -1.379204 -0.751334  0.149418 -0.052571 -0.044322   
       2022-04-30  -0.689209 -0.682024 -0.690625  0.592612  0.460036   

                       emb_5     emb_6     emb_7     emb_8     emb_9  ...  \
epk_id report_date                                                    ...   
0      2022-01-31  -0.139438  0.523077  0.549629  0.861828 -1.096254  ...   
       2022-02-28   0.028891 -0.408347  1.416915  0.614583 -1.343276  ...   
       2022-03-31  -0.258964 -1.497380 -1.173444 -0.946925 -0.346951  ...   
       2022-04-30  -0.134680 -0.325933  1.163237  0.407413 -1.624301  ...   
1      2022-01-31   0.572497 -0.575924  0.418597 -0.281893 -1.007878  ...   
...                      ...       ...       ...       ...       ...  ...   
468    2022-04-30   1.934619  0.615606  0.210141 -0.500605 -0.987975  ...   
469    2022-01-31   1.285198  0.269600 -0.151280 -0.287604 -1.209025  ...   
       2022-02-28   1.628728  0.150528 -0.115985  0.323152 -1.100011  ...   
       2022-03-31   1.673358  0.483628  0.054141 -0.424236 -1.016363  ...   
       2022-04-30   1.891775  0.222103  0.162234 -0.789787 -1.368718  ...   

                     emb_118   emb_119   emb_120   emb_121   emb_122  \
epk_id report_date                                                     
0      2022-01-31  -0.173302 -0.233520 -0.252624  1.033314 -0.004970   
       2022-02-28  -0.329221  0.869624 -0.420087  0.747626  0.311608   
       2022-03-31   0.047660 -0.904076  1.986186  0.927672 -0.575370   
       2022-04-30  -0.324072  0.286220  1.185525  0.307866  0.308692   
1      2022-01-31  -0.394580  0.350027  0.187081  0.841112  0.065283   
...                      ...       ...       ...       ...       ...   
468    2022-04-30  -1.094501  0.634504  0.358981  0.885834  0.161451   
469    2022-01-31  -0.943849  0.611137  0.640083  0.737641 -0.707990   
       2022-02-28  -1.154014  0.715447  0.170162  0.933860 -0.239987   
       2022-03-31  -1.215702  0.878712  0.316217  0.884946 -0.196047   
       2022-04-30  -1.046743  0.772128  0.356059  0.624043 -0.205632   

                     emb_123   emb_124   emb_125   emb_126   emb_127  
epk_id report_date                                                    
0      2022-01-31   1.166870 -0.818347  0.418107 -0.402436 -0.026494  
       2022-02-28   0.204995 -0.640759  2.361010 -0.341190 -0.244164  
       2022-03-31   1.242197 -0.537554  0.018201 -0.109680 -0.649192  
       2022-04-30   0.994041  0.406847  0.796898 -1.449584 -0.160539  
1      2022-01-31   0.188852 -1.233835  1.044441 -1.216439 -0.344071  
...                      ...       ...       ...       ...       ...  
468    2022-04-30   0.651174 -1.528779  0.491606 -0.737333  1.031261  
469    2022-01-31   0.902684 -1.478945  0.309499 -0.835328  0.378464  
       2022-02-28   0.382090 -1.271281  0.453382 -0.660094  0.295224  
       2022-03-31   0.309172 -1.569862  0.318319 -0.171513  0.643928  
       2022-04-30   0.517105 -1.352722  1.106019 -0.772372  0.731136  

[1852 rows x 128 columns]